## Pip installs

In [ ]:
!pip install torch

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [ ]:
!pip install pysam

In [ ]:
!pip install HTSeq

In [5]:
!pip install enformer-pytorch>=0.5

In [3]:
!pip install polars

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 30.8 MB/s eta 0:00:0000:0100:01


In [87]:
!pip install scikit-learn

## Module imports and variable definitions

In [1]:
import torch
import polars as pl
from enformer_pytorch import Enformer, GenomeIntervalDataset
from datasets import concatenate_datasets, load_dataset
import os
import numpy as np

/home/shawn/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
### define variables
hg38_bed_path = "/scratch/users/sschulz/pta_on_normal/Homo_sapiens_assembly38_n25chr.bed"
bam_dir= "/scratch/users/sschulz/pta_on_normal"
fasta_dir = bam_dir + '/fastas'
dataset_name='test_dataset.hf'
preprocess_bam=True
vcf_file=bam_dir +'/CARTPt04_Scan2_svc_merged_extract_snp.vcf.gz'
ref_fasta=bam_dir + '/Homo_sapiens_assembly38.fasta'
results_dir=bam_dir
output_fasta_fn='chr_test_fasta.fasta'
sbatch=0

## Function definitions

In [2]:
def preProcessVCF(vcf_path, ref_fasta, results_dir, output_fasta_fn, sbatch):
    '''
        take directory of a combined vcf file with somatic mutations called for normal cells
        with pta run on them and make a fasta file 
        
        TO-DO: make something that generates the vcf for you if given normal bam files
    '''

    !sh vcf2fasta.sh --vcf_path $vcf_path --ref_fasta $ref_fasta --results_dir $results_dir --output_name $output_fasta_fn --sbatch $sbatch

In [3]:
def makeEnformerDataset(fasta_path, bed_path):
    '''
        makes enformer dataset for genome intervals in bed file at bed_path, returns the dataset
    '''
    #filter_train = lambda df: df.filter(pl.col('column_4') == 'train')
    ds = GenomeIntervalDataset(
        bed_file = bed_path, ##<- this can just be the whole hg38.bed for all chromosomes
        fasta_file = fasta_path,  ## path to fasta file
        #filter_df_fn = filter_train,                        # filter dataframe function
        return_seq_indices = False,                          # return nucleotide indices (ACGTN) or one hot encodings
        shift_augs = (-2, 2),                               # random shift augmentations from -2 to +2 basepairs
        rc_aug = True,                                      # use reverse complement augmentation with 50% probability
        context_length = 196_608,
        return_augs = True                                  # return the augmentation meta data
        )
    return ds

In [4]:
def trainEnformerPTA(dataset):
    '''
        should train the enformer model
    '''
    
    #filter_train = lambda df: df.filter(pl.col('column_4') == 'train')

    model = HeadAdapterWrapper(
                enformer = dataset,
                num_tracks = 128,
                post_transformer_embed = False   # by default, embeddings are taken from after the final pointwise block w/ conv -> gelu - but if you'd like the embeddings right after the transformer block with a learned layernorm, set this to True
            ).cuda()

    return model

In [5]:
def makeOneHot(fastaFilePath, num_lines):
    '''
       Give path to a fasta file and the number of lines you want to turn into one hot encodings, get the one_hot encoded
       tensor
    '''
    with open(fastaFilePath, 'r') as input_file:
        try:
            head = [next(input_file).strip('\n') for _ in range(num_lines)]
        except StopIteration as e:
            print("Exceeded length of input file, stopping.")
    seq = str_to_one_hot(head)
    input_file.close()
    return(seq)
        

In [6]:
## this was all taken from lucidrains/enformer-pytorch, i have no clue if i could just source this somehow and i don't care

def torch_fromstring(seq_strs):
    def cast_list(t):
        return t if isinstance(t, list) else [t]
    batched = not isinstance(seq_strs, str)
    seq_strs = cast_list(seq_strs)
    np_seq_chrs = list(map(lambda t: np.fromstring(t, dtype = np.uint8), seq_strs))
    seq_chrs = list(map(torch.from_numpy, np_seq_chrs))
    return torch.stack(seq_chrs) if batched else seq_chrs[0]

def str_to_one_hot(seq_strs):
    seq_chrs = torch_fromstring(seq_strs)
    return one_hot_embed[seq_chrs.long()]

one_hot_embed = torch.zeros(256, 4)
one_hot_embed[ord('a')] = torch.Tensor([1., 0., 0., 0.])
one_hot_embed[ord('c')] = torch.Tensor([0., 1., 0., 0.])
one_hot_embed[ord('g')] = torch.Tensor([0., 0., 1., 0.])
one_hot_embed[ord('t')] = torch.Tensor([0., 0., 0., 1.])
one_hot_embed[ord('n')] = torch.Tensor([0., 0., 0., 0.])
one_hot_embed[ord('A')] = torch.Tensor([1., 0., 0., 0.])
one_hot_embed[ord('C')] = torch.Tensor([0., 1., 0., 0.])
one_hot_embed[ord('G')] = torch.Tensor([0., 0., 1., 0.])
one_hot_embed[ord('T')] = torch.Tensor([0., 0., 0., 1.])
one_hot_embed[ord('N')] = torch.Tensor([0., 0., 0., 0.])
one_hot_embed[ord('.')] = torch.Tensor([0.25, 0.25, 0.25, 0.25])

## Preprocessing

In [11]:
## preprocessing the target fasta
bam_dir='/home/shawn/datasets/enformer_data'
vcf_file=bam_dir +'/CARTPt04_Scan2_svc_merged_extract_snp.vcf.gz'
ref_fasta=bam_dir + '/Homo_sapiens_assembly38.fasta'
results_dir=bam_dir
output_fasta_fn='final_fasta.fasta'
preprocess_bam=False
if preprocess_bam:
    preProcessVCF(vcf_file, ref_fasta, results_dir, output_fasta_fn, sbatch)

In [9]:
## preprocessing the input fasta
vcf_file=bam_dir +'/CARTPt04_Scan2_svc_merged_extract_snp.vcf.gz'
ref_fasta=bam_dir + '/Homo_sapiens_assembly38.fasta'
results_dir=bam_dir
output_fasta_fn='test_fasta.fasta'
if preprocess_bam:
    preProcessVCF(vcf_file, ref_fasta, results_dir, output_fasta_fn, sbatch)

NameError: name 'bam_dir' is not defined

## get one-hot encodings with torch

In [27]:
lines_of_fasta_file=3000
targetFastaFilePath = bam_dir + '/final_fasta.fasta'

target = makeOneHot(targetFastaFilePath, lines_of_fasta_file)

/tmp/ipykernel_11198/318148619.py:8: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  np_seq_chrs = list(map(lambda t: np.fromstring(t, dtype = np.uint8), seq_strs))


In [28]:
lines_of_fasta_file=3000
inputFastaFilePath = bam_dir + '/test_fasta.fasta'

input_seq = makeOneHot(targetFastaFilePath, lines_of_fasta_file)

/tmp/ipykernel_11198/318148619.py:8: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  np_seq_chrs = list(map(lambda t: np.fromstring(t, dtype = np.uint8), seq_strs))


In [29]:
input_seq = input_seq

In [30]:
target = target

In [36]:
target.shape
input_seq.shape

torch.Size([3000, 60, 4])

## Run the model on new DNA tracks with somatic mutations detected in normal cells

In [32]:
import torch
from enformer_pytorch import Enformer, seq_indices_to_one_hot

# will need to figure out correct way to configure hyperparams
model = Enformer.from_hparams(
    dim = 1536,
    depth = 11,
    heads = 8,
    output_heads = dict(human = 4), #<- tbh idk what the correct number of output heads should be
    target_length = 1,
)


output, embeddings = model(input_seq, return_embeddings = True)

'''
    target should EITHER be the one hot encodings of an input fasta with many artifacts, or the weights from 
    just running the model on the one hot encodings without a target, not exactly sure which but try the 
    one hot encodigns first
    
    input_seq should be the one-hot embeddings of any fasta
    
    corr_coef should be higher if many artifacts, lower if not many artifacts. check against a bulk sequencing sample,
    another fasta that should have normal artifacts in it, and a single cell DNA sequencing sample
'''

loss, embeddings = model(
    input_seq,
    head = 'human',
    target = target,
    return_embeddings = True
)

loss.backward()

# after much training

corr_coef = model(
    input_seq,
    head = 'human',
    target = target,
    return_corr_coef = True
)

corr_coef # pearson R, used as a metric in the paper. For our first test, this should return something close to 1, since 
          # we are running it on the exact same fasta file as a sanity check


RuntimeError: The size of tensor a (4) must match the size of tensor b (5313) at non-singleton dimension 2

## Run the model with finetuning

In [16]:
import torch
from enformer_pytorch import Enformer
from enformer_pytorch.finetune import HeadAdapterWrapper

enformer = Enformer.from_pretrained('EleutherAI/enformer-official-rough')

model = HeadAdapterWrapper(
    enformer = enformer,
    num_tracks = 128,
    post_transformer_embed = False   # by default, embeddings are taken from after the final pointwise block w/ conv -> gelu - but if you'd like the embeddings right after the transformer block with a learned layernorm, set this to True
).cuda()

target = torch.randn(1, 1, 128).cuda()  # 128 tracks

loss = model(input_seq, target = target)
loss.backward()

In [18]:
model

HeadAdapterWrapper(
  (enformer): Enformer(
    (stem): Sequential(
      (0): Conv1d(4, 768, kernel_size=(15,), stride=(1,), padding=(7,))
      (1): Residual(
        (fn): Sequential(
          (0): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (1): GELU()
          (2): Conv1d(768, 768, kernel_size=(1,), stride=(1,))
        )
      )
      (2): AttentionPool(
        (pool_fn): Rearrange('b d (n p) -> b d n p', p=2)
        (to_attn_logits): Conv2d(768, 768, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
    )
    (conv_tower): Sequential(
      (0): Sequential(
        (0): Sequential(
          (0): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (1): GELU()
          (2): Conv1d(768, 768, kernel_size=(5,), stride=(1,), padding=(2,))
        )
        (1): Residual(
          (fn): Sequential(
            (0): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stat

In [20]:
pred = model(input_seq, head = 'human') # (3000, 3000)

TypeError: HeadAdapterWrapper.forward() got an unexpected keyword argument 'head'

In [21]:
pred

tensor([[[0.7276, 0.7898, 0.7651,  ..., 0.8316, 0.6573, 0.5948]],

        [[0.7207, 0.7824, 0.8117,  ..., 0.7090, 0.5889, 0.5795]],

        [[0.5686, 0.8286, 0.8646,  ..., 0.7037, 0.6471, 0.6555]],

        ...,

        [[0.7175, 0.7805, 0.7743,  ..., 0.7038, 0.8123, 0.5317]],

        [[0.6933, 0.6910, 0.7489,  ..., 0.7125, 0.6352, 0.6702]],

        [[0.6963, 0.7716, 0.8335,  ..., 0.6973, 0.6662, 0.6498]]],
       grad_fn=<SoftplusBackward0>)

## Old stuff

In [ ]:
dataset_list = []
fai_dir=fasta_dir + '/fais'
!mkdir -p {fai_dir}
for fasta_name in os.listdir(fasta_dir):
    !mv {fasta_dir}/*.fai {fai_dir}
    fasta_path=fasta_dir + '/' + fasta_name
    if not os.path.splitext(fasta_path)[1] == 'fai':
        print('Now collecting dataset for: ' + fasta_path)
        dataset_list.append(makeEnformerDataset(fasta_path, hg38_bed_path))

mv: cannot stat ‘/scratch/users/sschulz/pta_on_normal/fastas/*.fai’: No such file or directory
Now collecting dataset for: /scratch/users/sschulz/pta_on_normal/fastas/CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-A9_S57.realigned_deduped_sorted.fasta
Now collecting dataset for: /scratch/users/sschulz/pta_on_normal/fastas/CART-MRD-BALL-PTA-NEXTERA-WGS-CCT5007Pt04-B10_S65.realigned_deduped_sorted.fasta


In [8]:
def combineDatasets(dataset_list):
    '''
        uses hf datasets to concatenate a list of datasets
    '''
    return concatenate_datasets(dataset_list)

In [ ]:
combined_dataset = combineDatasets(dataset_list)

In [ ]:
combined_dataset.save_to_disk(bam_dir + '/' + dataset_name)

In [ ]:
model = trainEnformerPTA(combined_datset)

In [ ]:
seq = ### enter a sequence to test
pred = model(seq, head = 'human') # (896, 5313) ###<- what does human mean here?